In [16]:
import os
import sys
import torch
import speechbrain as sb
from hyperpyyaml import load_hyperpyyaml
from speechbrain.utils.parameter_transfer import Pretrainer

from models.ADA_IE import ADA_IE
from datasets.ADAIE_dataset import get_dataset

In [27]:
import speechbrain as sb
import os
import json


LABEL_DIR = '/media/beck/ssd/datasets_opensource/sr/processed_data'
SPEAKER_ID_LIST = 'speaker_id_list_train_dc.json'
AG_LIST = 'AG_list.json'
C_ID_LIST = 'c_id_list.json'

In [28]:
label_encoder_spk = sb.dataio.encoder.CategoricalEncoder()
label_encoder_ag = sb.dataio.encoder.CategoricalEncoder()
label_encoder_c = sb.dataio.encoder.CategoricalEncoder()

In [29]:
@sb.utils.data_pipeline.takes("path") # file_path
@sb.utils.data_pipeline.provides("sig")
def audio_pipeline(path):
	sig = sb.dataio.dataio.read_audio(path)
	return sig

# Define label pipeline:
@sb.utils.data_pipeline.takes("VoxCeleb_ID") # speaker_id
@sb.utils.data_pipeline.provides("speaker_id", "speaker_encoded")
def speaker_label_pipeline(speaker_id):
	yield speaker_id
	speaker_encoded = label_encoder_spk.encode_label_torch(speaker_id, True)
	yield speaker_encoded

# Define label pipeline:
@sb.utils.data_pipeline.takes("agegroup")
@sb.utils.data_pipeline.provides("agegroup", "agegroup_encoded")
def agegroup_label_pipeline(agegroup):
	yield agegroup
	agegroup_encoded = label_encoder_ag.encode_label_torch(agegroup, True)
	yield agegroup_encoded

@sb.utils.data_pipeline.takes("child_id")
@sb.utils.data_pipeline.provides("child_id", "c_encoded")
def child_id_label_pipeline(child_id):
	yield child_id
	c_encoded = label_encoder_c.encode_label_torch(child_id, True)
	yield c_encoded

In [30]:
# hparams_file, run_opts, overrides = sb.parse_arguments(sys.argv[1:])
hparams_file = "./yaml/ADA_IE.yaml"
# sb.utils.distributed.ddp_init_group(run_opts)

with open(hparams_file) as fin:
	hparams = load_hyperpyyaml(fin)  # , overrides

/media/beck/ssd/datasets_opensource/sr/augment/rirs_noises.zip exists. Skipping download


In [31]:
def load_create_label_encoder(label_enc_file,
								label_encoder,
								label_list_file = None,
								):
	label_list = None
	
	with open(os.path.join(LABEL_DIR, label_list_file), 'r') as f:
		label_list = [tuple(json.load(f))]

	lab_enc_file = os.path.join(hparams["save_folder"], label_enc_file)
	label_encoder.load_or_create(
		path=lab_enc_file,
		sequence_input=False,
		from_iterables=label_list,
	)

In [32]:
datasets = {}

for dataset in ["train", "dev", "eval"]:
	datasets[dataset] = sb.dataio.dataset.DynamicItemDataset.from_json(
		json_path=hparams[f"{dataset}_annotation"],
		replacements={"data_root": hparams["data_folder"]},
		dynamic_items=[
						audio_pipeline,
						speaker_label_pipeline,
						agegroup_label_pipeline,
						child_id_label_pipeline,
					#    triplet_label_pipeline
						],
		output_keys=["id", "sig",
					"speaker_id", "speaker_encoded",
					"agegroup", "agegroup_encoded",
					"child_id", "c_encoded", 
					# "triplet_encoded"
						],
	)

In [24]:
load_create_label_encoder(label_enc_file="label_encoder_speaker.txt",
							label_encoder=label_encoder_spk,
							label_list_file=SPEAKER_ID_LIST
						)
label_encoder_spk.add_unk()

2741

In [33]:
load_create_label_encoder(label_enc_file="label_encoder_ag.txt",
                            label_encoder=label_encoder_ag,
                            label_list_file=AG_LIST
                        )
label_encoder_ag.add_unk()

3

In [26]:
def load_create_label_encoder(label_enc_file,
								label_encoder,
								label_list_file = None,
								):
	label_list = None
	
	with open(os.path.join(LABEL_DIR, label_list_file), 'r') as f:
		label_list = [tuple(json.load(f))]

	lab_enc_file = os.path.join(hparams["save_folder"], label_enc_file)
	label_encoder.load_or_create(
		path=lab_enc_file,
		sequence_input=False,
		from_iterables=label_list,
	)

In [1]:
import os
import sys
import torch
import speechbrain as sb
from hyperpyyaml import load_hyperpyyaml
from speechbrain.utils.parameter_transfer import Pretrainer

from models.ADA_IE import ADA_IE
from datasets.ADAIE_dataset import get_dataset

/home/beck/.pyenv/versions/3.7.10/envs/ada/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
torch.cuda.empty_cache()

In [3]:
# hparams_file, run_opts, overrides = sb.parse_arguments(sys.argv[1:])
hparams_file = "./yaml/ADA_IE.yaml"
# sb.utils.distributed.ddp_init_group(run_opts)

with open(hparams_file) as fin:
	hparams = load_hyperpyyaml(fin)  # , overrides

/media/beck/ssd/datasets_opensource/sr/augment/rirs_noises.zip exists. Skipping download


/home/beck/.pyenv/versions/3.7.10/envs/ada/lib/python3.7/site-packages/torch/nn/modules/rnn.py:63: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [9]:
pretrain = Pretrainer(loadables={'model': hparams['modules']['fbanks_encoder']},
						paths={'model': "speechbrain/spkrec-ecapa-voxceleb/embedding_model.ckpt"})

In [7]:
pretrain.collect_files()
pretrain.load_collected()

TypeError: cached_download() takes 1 positional argument but 2 were given